In [1]:
num_elements = 100

In [2]:
import pandas as pd
from faker import Faker
fake = Faker()

In [3]:
import numpy as np
elements = np.array(["John", "James", "Alex", "Oliver", "Matt", "Tom", "Luke"])
probabilites = np.array([0.4,0.2,0.1,0.1,0.1,0.05,0.05])
male_first_names = np.random.choice(elements, int(num_elements*0.8), list(probabilites))
males = male_first_names.copy()
males[:] = "Male"

elements = np.array(["Emily", "Emma", "Sophie", "Hannah", "Kate", "Rebecca", "Chloe"])
female_first_names = np.random.choice(elements, int(num_elements*0.2), list(probabilites))
females = female_first_names.copy()
females[:] = "Female"

first_names = np.concatenate([male_first_names, female_first_names])

genders = np.concatenate([males,females])

In [4]:
elements = np.array(["Smith",
"Jones",
"Williams",
"Brown",
"Taylor",
"Davies",
"Wilson",
"Evans",
"Thomas",
"Johnson",
"Roberts",
"Walker",
"Wright",
"Thompson",
"Robinson",
"White",
"Hughes",
"Edwards",
"Hall",
"Green",
"Martin",
"Wood"])
surnames = np.random.choice(elements, num_elements)

In [5]:
elements = np.array(["London",
"Birmingham",
"Leeds",
"Glasgow",
"Sheffield",
"Bradford",
"Liverpool",
"Edinburgh",
"Manchester",
"Bristol",
"Kirklees",
"Fife"])
cities = np.random.choice(elements, num_elements)

In [6]:
target_dataset = pd.DataFrame([genders, first_names, surnames, cities]).T
target_dataset.columns = ["gender", "first_name", "surname", "city"]

In [7]:
for r in target_dataset.iterrows():
    index = r[0]
    target_dataset.loc[index,"dob"] =  fake.date_time_between(start_date="-30y", end_date="now", tzinfo=None).strftime("%Y-%b-%d")
    target_dataset.loc[index,"code"] = fake.md5()

In [8]:
target_dataset.head()

,gender,first_name,surname,city,dob,code
0,Male,Alex,Wright,Edinburgh,2003-May-16,daec4bd1f44b36fd66992221c7ebd40d
1,Male,Oliver,Hughes,Leeds,2000-Jun-15,d000ef00067505bade11a055cec38c67
2,Male,Matt,Thompson,Bristol,1986-Oct-27,23ffae4d943bdd340a81f4e7b3aaec62
3,Male,Alex,Evans,Glasgow,2009-Jun-23,9b837b1ada9dc6129f799bd718292c98
4,Male,Alex,Green,Kirklees,1988-Feb-01,80972206fa6c585f22090d7f19f5d310


In [9]:
target_dataset = target_dataset.join(target_dataset.dob.str.split("-",expand=True))
target_dataset = target_dataset.rename(columns={0:"year", 1:"month", 2:"day"})
target_dataset = target_dataset.drop("dob", axis=1)
target_dataset.head()

,gender,first_name,surname,city,code,year,month,day
0,Male,Alex,Wright,Edinburgh,daec4bd1f44b36fd66992221c7ebd40d,2003,May,16
1,Male,Oliver,Hughes,Leeds,d000ef00067505bade11a055cec38c67,2000,Jun,15
2,Male,Matt,Thompson,Bristol,23ffae4d943bdd340a81f4e7b3aaec62,1986,Oct,27
3,Male,Alex,Evans,Glasgow,9b837b1ada9dc6129f799bd718292c98,2009,Jun,23
4,Male,Alex,Green,Kirklees,80972206fa6c585f22090d7f19f5d310,1988,Feb,01


In [10]:
# Now we need to create a corrupted version of this for matching
candidate_dataset = target_dataset.sample(frac=0.8,replace=False).copy()

In [11]:
candidate_dataset.head()

,gender,first_name,surname,city,code,year,month,day
54,Male,Matt,Evans,Leeds,e354be655f29a2fcad1ca05dedf653f9,2004,Nov,13
75,Male,John,Green,Birmingham,8b2e048dd457fe93d442e735abf22df8,1987,Oct,30
88,Female,Emily,Green,Liverpool,3ac9302bddfb4d222acc672eb6c3eb25,2008,Jun,12
35,Male,John,Thompson,Birmingham,98182e6f9e70c6762d781f832a553b88,1999,Sep,03
17,Male,Alex,Brown,Bradford,ccd8004da51b3d5c4d3db37802cb5983,2015,May,07


In [12]:
import random
import string


def switch(my_string):
    pos1 = random.randrange(0,len(my_string)) 
    pos2 = random.randrange(0,len(my_string)) 
    my_string = list(my_string)
    char1 = my_string[pos1]
    char2 = my_string[pos2]
    my_string[pos1] = char2
    my_string[pos2] = char1
    return "".join(my_string)

def new_letter(my_string):
    pos1 = random.randrange(0,len(my_string)) 
    letter = random.choice(string.ascii_lowercase)
    return my_string[:pos1] + letter + my_string[pos1:]

def delete_letter(my_string):
    pos1 = random.randrange(1,len(my_string)) 
    return my_string[:pos1] + my_string[pos1+1:]

def corrupt_string(my_string, num_switches=1,num_new_letters=0,num_deletes=0):
    
    for i in range(num_switches):
        my_string = switch(my_string)
        
    for i in range(num_new_letters):
        my_string = new_letter(my_string)
        
    for i in range(num_deletes):
        my_string = delete_letter(my_string)
        
    return my_string

In [13]:
for r in candidate_dataset.iterrows():
    index = r[0]
    row = r[1]
    
    if (random.random()>0.9):
        candidate_dataset.loc[index, "first_name"] = corrupt_string(row["first_name"])
    
    if (random.random()>0.9):
        candidate_dataset.loc[index, "surname"] = corrupt_string(row["surname"], num_switches=0, num_deletes=1)
        
    if (random.random()>0.9):
        candidate_dataset.loc[index, "city"] = corrupt_string(row["city"], num_switches=0, num_deletes=1)
        
    if (random.random()>0.9):
        surname = row["surname"]
        first_name = row["first_name"]
        candidate_dataset.loc[index, "first_name"] = surname
        candidate_dataset.loc[index, "surname"] = first_name
        
    if (random.random()>0.9):
        candidate_dataset.loc[index,"year"] = fake.date_time_between(start_date="-30y", end_date="now", tzinfo=None).strftime("%Y")
        
    if (random.random()>0.9):
        candidate_dataset.loc[index,"day"] = fake.date_time_between(start_date="-30y", end_date="now", tzinfo=None).strftime("%d")
    
    if (random.random()>0.8):
        candidate_dataset.loc[index, "gender"] = None
        
    if (random.random()>0.8):
        candidate_dataset.loc[index, "first_name"] = None
        
    if (random.random()>0.8):
        candidate_dataset.loc[index, "surname"] = None
        
    if (random.random()>0.8):
        candidate_dataset.loc[index, "year"] = None
        
    if (random.random()>0.8):
        candidate_dataset.loc[index, "month"] = None
        
    if (random.random()>0.8):
        candidate_dataset.loc[index, "day"] = None
        

        

In [14]:
target_dataset.to_csv("target_dataset.csv", encoding="utf-8", index=False)

In [15]:
candidate_dataset.to_csv("candidate_dataset.csv", encoding="utf-8", index=False)